The workflow for running the NFDRS Calculator in the Command Line Interface is:
1. Load station metadata .csv
2. Generate an initialization file (from InitTemplate.cfg) with station-specific information
3. (More pre-processing to get to aggregate weather data .fw21)
4. Generate station-specific configuration file (from ConfigTemplate.cfg) with
      - the station-specific initialization file
      - the weather data .fw21
5. Run the NFDRS using the configuration file
6. Outputs an AllOutputs .csv 

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess, sys

button = widgets.Button(description="Setup Code")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        try:
            # surpress package installation
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numpy", "pandas", "matplotlib"])
            
            # --- import after install ---
            import os
            from pathlib import Path
            import pandas as pd
            from datetime import datetime
            from tqdm.notebook import tqdm
            
            print("Setup complete")
        except Exception as e:
            print("Setup failed:", e)

button.on_click(on_button_clicked)
display(button, output)


Button(description='Setup Code', style=ButtonStyle())

Output()

In [2]:
# Install ipyfilechooser if needed
import subprocess, sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ipyfilechooser"])

# Imports
import os
from pathlib import Path
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output
import ipywidgets as widgets
import pandas as pd

# Helper functions
def find_target_folders(root_dir: str, target_folder: str):
    """Return a list of Paths to folders whose basename matches target_folder (case-insensitive)."""
    matches = []
    target_norm = target_folder.strip().casefold()
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if Path(dirpath).name.strip().casefold() == target_norm:
            matches.append(Path(dirpath))
    return matches

def list_all_files_recursive(folder: Path):
    """Return a list[Path] of all files under folder, recursively."""
    out = []
    for dp, dn, fns in os.walk(folder):
        for fn in fns:
            out.append(Path(dp) / fn)
    return out

# File chooser for root folder
chooser = FileChooser(Path.home(), title="Select a directory to search for the working folder")
chooser.show_only_dirs = True
button = widgets.Button(description="Verify components")
output = widgets.Output()

target_folder = "NFDRS4_Local_Template"  # standardized folder name

def search_folders(b):
    with output:
        clear_output()
        root_dir = chooser.selected_path
        if not root_dir:
            print("No folder selected. Please select a root folder first.")
            return
        
        # Find target folders
        folders = find_target_folders(root_dir, target_folder)
        if not folders:
            print(f"No folder named '{target_folder}' found under {root_dir}")
            return
        
        # Collect all files
        files_by_folder = {fld: list_all_files_recursive(fld) for fld in folders}
        all_files = [p for plist in files_by_folder.values() for p in plist]
        
        # Required file paths
        candidates = {
            "nfdrs4_path": {"nfdrs4_cli.exe"},
            "config_template": {"configtemplate.cfg", "config_template.cfg"},
            "init_template": {"inittemplate.cfg", "init_template.cfg"},
        }

        resolved = {k: None for k in candidates}
        for p in all_files:
            name = p.name.casefold()
            for key, names in candidates.items():
                if name in names and resolved[key] is None:
                    resolved[key] = p

        # Validate
        missing_keys = [k for k, v in resolved.items() if v is None]
        if missing_keys:
            preview = "\n".join(str(p) for p in sorted(all_files)[:25])
            details = "; ".join(f"{k}: expected one of {sorted(list(candidates[k]))}" for k in missing_keys)
            print("Could not locate all required files:")
            print(f"Missing -> {details}")
            print(f"Searched under {len(folders)} folder(s) named '{target_folder}':")
            for f in folders:
                print(f" - {f}")
            if all_files:
                print("\nHere are some files I did see (first 25):")
                print(preview)
            return
        
        # Bind variables for next steps
        nfdrs4_path = resolved["nfdrs4_path"]
        config_template = resolved["config_template"]
        init_template = resolved["init_template"]
        
        # Set working directory
        working_dir = folders[0]
        os.chdir(working_dir)
        
        # Output results
        print("Selected folder set as workspace")

# Bind button
button.on_click(search_folders)

# Display chooser and button
display(chooser, button, output)


FileChooser(path='C:\Users\tracy.tien', filename='', title='Select a directory to search for the working folde…

Button(description='Verify components', style=ButtonStyle())

Output()

In [3]:
# ----- User select area of interest for analysis (i.e. station, all stations, PSA)

# Load weather station metadata
tx_wx_stations = working_dir / "TX_Stations_Full.csv"
df = pd.read_csv(tx_wx_stations)

# Prepare station dropdown (display name but return ID)
station_options = [
    (row["StationName"], row["StationID"]) for _, row in df.iterrows()
]
station_options = sorted(station_options, key=lambda x: x[0])

# Prepare PSA dropdown
psa_options = sorted(df["PSA"].dropna().unique().tolist())

# Parent dropdown to select area of interest type
aoi_type_dropdown = widgets.Dropdown(
    options=["All Stations", "Single Station", "PSA"],
    description="Area of Interest:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="40%")
)

# Station dropdown
station_dropdown = widgets.Dropdown(
    options=station_options,
    description="Station Name:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="50%")
)

# PSA dropdown
psa_dropdown = widgets.Dropdown(
    options=psa_options,
    description="PSA:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="30%")
)

# Output areas
aoi_output = widgets.Output()
result_output = widgets.Output()

# Global variable to store selected IDs
selected_ids = []

# Update dropdown visibility depending on AOI selection
def update_aoi_dropdown(change):
    with aoi_output:
        aoi_output.clear_output()
        if change["new"] == "Single Station":
            display(station_dropdown)
        elif change["new"] == "PSA":
            display(psa_dropdown)
        # If "All Stations" → no extra dropdown

aoi_type_dropdown.observe(update_aoi_dropdown, names="value")

# Confirm selection
run_button = widgets.Button(
    description="Confirm Selection",
    button_style="success",
    layout=widgets.Layout(width="200px")
)

# Logic when button is clicked
def on_run_clicked(b):
    global selected_ids
    with result_output:
        result_output.clear_output()
        aoi_type = aoi_type_dropdown.value

        if aoi_type == "All Stations":
            selected_ids = df["StationID"].tolist()

        elif aoi_type == "Single Station":
            selected_ids = [station_dropdown.value]
            preview = df[df["StationID"].isin(selected_ids)][["StationName", "StationID"]]
            display(preview)

        elif aoi_type == "PSA":
            selected_psa = psa_dropdown.value
            selected_ids = df.loc[df["PSA"] == selected_psa, "StationID"].tolist()
            preview = df[df["StationID"].isin(selected_ids)][["StationName", "StationID"]]
            display(preview)

run_button.on_click(on_run_clicked)

# Display widgets
display(aoi_type_dropdown, aoi_output, run_button, result_output)


Dropdown(description='Area of Interest:', layout=Layout(width='40%'), options=('All Stations', 'Single Station…

Output()

Button(button_style='success', description='Confirm Selection', layout=Layout(width='200px'), style=ButtonStyl…

Output()

In [5]:
# ----- User run CLI for selection ----- 

# Setup log file 
log_file = working_dir / f"workflow_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

# Helper function to log messages
def log_message(message):
    with open(log_file, "a") as f:
        f.write(message + "\n")
    print(message)

# Workflow function for a single station
def run_station_workflow(station_id):
    """
    Run all steps for a single station:
    1. Create initialization cfg
    2. Download weather data
    3. Create configuration cfg
    4. Run CLI
    """
    try:
        log_message(f"--- Starting station {station_id} ---")
        
        # -----------------------------
        # Step 1: create station-specific initialization .cfg files. In FF+, this is equal to setting the SIG/Station under Active Working Set Definition.
        template_txt = init_template.read_text()
        row = df[df["StationID"] == station_id].iloc[0]
        
        # Generate CFG text with replacements
        cfg_text = template_txt
        replacements = {
            "LATITUDE": str(row["latitude"]),
            "ANNUALPRECIP": str(row["avgAnnualPrecip"]),
            "VPDMIN": str(row["gsiVpdMin"]),
            "VPDMAX": str(row["gsiVpdMax"])
        }
        for key, val in replacements.items():
            cfg_text = cfg_text.replace(f"<{key}>", val)
        
        # Create a subfolder to store initialization files
        init_dir = working_dir / "Init"
        init_dir.mkdir(exist_ok=True)
        cfg_path = init_dir / f"Init_{station_id}.cfg"
        cfg_path.write_text(cfg_text)
        station_cfg_path = cfg_path
        
        print(f"Generated initialization file for station {station_id} at {cfg_path}")
        log_message(f"Generated initialization file for station {station_id} at {cfg_path}")
        
        # -----------------------------
        # Step 2: Donwload weather data from the server

        import requests
        
        psa_fw21_urls = {
            "Caprock": "https://twcgis.tamu.edu/fw21/PSA/Caprock.fw21",
            "Central Texas": "https://twcgis.tamu.edu/fw21/PSA/Central_Texas.fw21",
            "Cross Timnbers": "https://twcgis.tamu.edu/fw21/PSA/Cross_Timbers.fw21",
            "Eastern Hill Country": "https://twcgis.tamu.edu/fw21/PSA/Eastern_Hill_Country.fw21",
            "High Plains": "https://twcgis.tamu.edu/fw21/PSA/High_Plains.fw21",
            "Lower Gulf Coast": "https://twcgis.tamu.edu/fw21/PSA/Lower_Gulf_Coast.fw21",
            "North Texas": "https://twcgis.tamu.edu/fw21/PSA/North_Texas.fw21",
            "Rolling Plains": "https://twcgis.tamu.edu/fw21/PSA/Rolling_Plains.fw21",
            "South Texas": "https://twcgis.tamu.edu/fw21/PSA/Southeast_Texas.fw21",
            "Southeast Texas": "https://twcgis.tamu.edu/fw21/PSA/Southeast_Texas.fw21",
            "Southern Plains": "https://twcgis.tamu.edu/fw21/PSA/Southern_Plains.fw21",
            "Trans Pecos": "https://twcgis.tamu.edu/fw21/PSA/Trans_Pecos.fw21",
            "Upper Gulf Coast": "https://twcgis.tamu.edu/fw21/PSA/Upper_Gulf_Coast.fw21",
            "Western Hill Country": "https://twcgis.tamu.edu/fw21/PSA/Western_Hill_Country.fw21",
            "Western Pineywoods": "https://twcgis.tamu.edu/fw21/PSA/Western_Pineywoods.fw21"
        }
        
        # Get the selected station
        row = df[df["StationID"] == station_id].iloc[0]
        
        # Lookup the station's PSA
        station_psa = row["PSA"]
        print(f"Station {station_id} belongs to PSA {station_psa}")
        
        # Determine the FW21 URL for this PSA
        fw21_url = psa_fw21_urls.get(station_psa)
        if fw21_url is None:
            raise ValueError(f"No FW21 URL defined for PSA {station_psa}")
        
        # Download FW21 file
        fw21_dir = working_dir / "Daily_FW21"
        fw21_dir.mkdir(exist_ok=True)
        fw21_file = fw21_dir / f"FW21_{station_psa}.fw21"
        wx_log_dir = fw21_dir / "Wx_Log"
        
        if not fw21_file.exists():
            print(f"Downloading FW21 file for PSA {station_psa}...")
            resp = requests.get(fw21_url)
            resp.raise_for_status()
            fw21_file.write_bytes(resp.content)
            print(f"Saved FW21 file to {fw21_file}")
        else:
            print(f"Using cached FW21 file at {fw21_file}")
        
        # [9/3/2025 NOTE: The CLI threw errors/warnings of blank values in the required variables, this is just to log and ignore the bad rows so this notebook can run] 
        df_fw21 = pd.read_csv(fw21_file)
        required_cols = ["SolarRadiation(W/m2)", "Temperature(F)", "WindSpeed(mph)", "WindAzimuth(degrees)"]
        bad_rows = pd.DataFrame()
        for col in required_cols:
            if col in df_fw21.columns:
                bad_rows = pd.concat([bad_rows, df_fw21[df_fw21[col].isna() | (df_fw21[col]==-999)]])
        bad_rows = bad_rows.drop_duplicates()
        if not bad_rows.empty:
            wx_log_dir.mkdir(exist_ok=True)
            log_file = wx_log_dir / f"fw21_badrows_{fw21_file.stem}.log"
            with open(log_file, "a") as f:
                f.write(f"==== Log generated {datetime.now().isoformat()} ====\n")
                f.write(f"FW21 file: {fw21_file}\n")
                f.write(f"Total bad rows: {len(bad_rows)}\n\n")
                bad_rows.to_csv(f, index=False, header=True)
            print(f"Found {len(bad_rows)} bad rows, logged to {log_file}")
        else:
            print(f"No bad rows detected in {fw21_file}")

        log_message(f"Downloaded weather data for {station_id}")
        
        # -----------------------------
        # Step 3: Create station-specific confirguration file
        def generate_config_file(
            station_id,
            init_file: str,
            fw21_file: str,
            load_state: str = "",
            save_state: str = "",
            outputs_file: str = "",
            index_file: str = "",
            stored_outputs: str = "0",
            config_dir: Path = working_dir / "Config"
        ):
            """
            Generate a Configuration CFG file for NFDRS4 CLI.
            """
            config_dir.mkdir(exist_ok=True)
            
            template = f"""initFile = "{init_file}";
        # required as input for processing
        wxFile = "{fw21_file}";
        #NFDRSState saving and loading capabilities (optional)
        #loadFromState will load the state file and begin any calculations from the saved state
        loadFromStateFile = "{load_state}";
        #saveToStateFile will save the state when calculation is complete to the indicated file
        saveToStateFile = "{save_state}";
        # output files (csv) can be designated, otherwise nothing is output 
        # if they exist, they are appended to by the program
        # output files include a date/time and selected outputs
        #csv header is only written to the output file if the file is being created
        #all outputs available, includes fuel moistures and indexes
        allOutputsFile = "{outputs_file}";
        #indexes only
        indexOutputFile = "{index_file}";
        #fuel moistures
        fuelMoisturesOutputFile = "";
        #outputInterval 0 = hourly (each record), 1 = daily (at ObsHour from NFDRSInit file)
        outputInterval = "0";
        #Option to use stored outputs from previously run 'allOutputsFile' output 
        #causes NFDRS4 to bypass running of Nelson and GSI models, calculating indexes using stored NFDRS4 outputs
        #an error will be returned if 'wxFile' does not contain necessary fields
        #any record missing necessary any fields will be skipped
        useStoredOutputs = "{stored_outputs}";
        #Added as required, 4/27/2024
        #to accomodate multiple stations in a single FW21 format file
        #stationID was added as a data element to FW21 and NFDRS4_cli config file
        #this stationID will be used when StationID is not present in FW21
        stationID = "{station_id}";
        """
        
            cfg_path = config_dir / f"Config_{station_id}.cfg"
            cfg_path.write_text(template)
            print(f"Generated configuration CFG for station {station_id} at {cfg_path}")
            return cfg_path
        
        outputs_dir = working_dir / "Outputs"
        outputs_dir.mkdir(exist_ok=True)
        
        init_file = working_dir / f"Init/Init_{station_id}.cfg"
        fw21_file = working_dir / f"Daily_FW21/FW21_{df[df['StationID']==station_id]['PSA'].values[0]}.fw21"
        outputs_file = working_dir / f"Outputs/AllOutputs_{station_id}.csv"
        index_file = working_dir / f"Outputs/Index_{station_id}.csv"
        
        config_file = generate_config_file(
            station_id,
            init_file=str(init_file),
            fw21_file=str(fw21_file),
            load_state="",
            save_state="",
            outputs_file=str(outputs_file),
            index_file=str(index_file),
            stored_outputs="0"
        )

        log_message(f"Generated configuration CFG for station {station_id} at {cfg_path}")
        
        # -----------------------------
        # Step 4: Run CLI
        import subprocess
        
        def run_ffp_cli(config_file: Path, cli_exe: Path = working_dir / "NFDRS4_cli.exe", cli_log_dir: Path = None):
            """
            Runs the NFDRS4 CLI executable with the specified configuration file.
            Prints stdout and stderr, and returns the subprocess.CompletedProcess object.
            """
            if not cli_exe.exists():
                raise FileNotFoundError(f"CLI executable not found at {cli_exe}")
        
            if not config_file.exists():
                raise FileNotFoundError(f"Configuration file not found at {config_file}")
        
            print(f"Running NFDRS CLI for config: {config_file}")
            result = subprocess.run(
                [str(cli_exe), str(config_file)],
                capture_output=True,
                text=True,
                shell=True
            )
        
            cli_log_dir = working_dir / "Log"
            cli_log_dir.mkdir(exist_ok=True)
        
            cli_log_file = cli_log_dir / F"{config_file.stem}_log.txt"
            
            with cli_log_file.open("w", encoding="utf-8") as f:
                f.write("---- STDOUT ----\n")
                f.write(result.stdout)
                f.write("\n---- STDERR ----\n")
                f.write(result.stderr)
                f.write(f"\nReturn code: {result.returncode}\n")
        
            print("---- STDOUT ----")
            print(result.stdout[:1000])  # print first 1000 chars for preview
            print("---- STDERR ----")
            print(result.stderr[:1000])
            print(f"Return code: {result.returncode}")
        
            if result.returncode != 0:
                print(f"CLI returned an error for {config_file}")
        
            return result

        init_file = working_dir / f"Init/Init_{station_id}.cfg"
        config_file = working_dir / f"Config/Config_{station_id}.cfg"
        cli_exe = working_dir / "NFDRS4_cli.exe"
        
        result = run_ffp_cli(config_file, cli_exe)
        # -----------------------------
        
        log_message(f"--- Finished station {station_id} ---\n")
        
    except Exception as e:
        log_message(f"Error processing station {station_id}: {e}")

# ------------------------
# Button to run workflow for all selected stations
# ------------------------
run_all_button = widgets.Button(
    description="Run Workflow",
    button_style="primary",
    layout=widgets.Layout(width="200px")
)

run_output = widgets.Output()

def on_run_all_clicked(b):
    with run_output:
        run_output.clear_output()
        if not selected_ids:
            print("No stations selected. Please select an area of interest first.")
            return
        
        print(f"Running workflow for {len(selected_ids)} station(s)...")
        
        # Use tqdm progress bar
        for station_id in tqdm(selected_ids, desc="Stations", unit="station"):
            run_station_workflow(station_id)
        
        print(f"\nWorkflow completed! Log saved to:\n{log_file}")

run_all_button.on_click(on_run_all_clicked)

display(run_all_button, run_output)


Button(button_style='primary', description='Run Workflow', layout=Layout(width='200px'), style=ButtonStyle())

Output()